In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-04-02 18:45:32.283636


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
dfLitt = pd.json_normalize(data["data"])

In [3]:
dfLittShowTrue = dfLitt[dfLitt["show"] == True].copy()
dfLittShowTrue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3790 entries, 2 to 4882
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   authorid                       3790 non-null   object 
 1   authorid_norm                  3790 non-null   object 
 2   db_checksum                    3790 non-null   object 
 3   db_timestamp                   3790 non-null   int64  
 4   doc_type                       3790 non-null   object 
 5   full_name                      3790 non-null   object 
 6   gender                         3790 non-null   object 
 7   imported                       3361 non-null   object 
 8   intro                          723 non-null    object 
 9   name_for_index                 3790 non-null   object 
 10  pictureinfo                    247 non-null    object 
 11  searchable                     3790 non-null   bool   
 12  show                           3790 non-null   b

In [ ]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# 
query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID)  ?authorid ?SBL ?SKBL WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P5101 ?authorid
OPTIONAL {?item wdt:P3217 ?SBL}
OPTIONAL {?item wdt:P4963 ?SKBL}
} order by ?authorid"""
 
    
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittbanktot = get_sparql_dataframe(endpoint_url, query)

In [ ]:
WDLittbanktot

In [ ]:
# df = pd.merge(dfA, dfB, on=['a','b'], how="outer", indicator=True)
WDLittbank_WD_merge=  pd.merge(dfLittShowTrue,WDLittbanktot, how="outer", on='authorid',indicator=True)
WDLittbank_WD_merge.rename(columns={"_merge": "WD_Littbank_merge_Outer"},inplace = True)

WDLittbank_WD_merge["WD_Littbank_merge_Outer"].value_counts()



In [ ]:
WDLittbank_WD_merge_Not_connected = \
WDLittbank_WD_merge[WDLittbank_WD_merge["WD_Littbank_merge_Outer"] == "left_only"].copy()

In [ ]:
WDLittbank_WD_merge_Not_connected.head()

In [ ]:
WDLittbank_WD_merge_Not_connected[WDLittbank_WD_merge_Not_connected['librisid']. 
                                  notna()][["authorid","librisid","show","full_name"]]

In [ ]:
WDLittbank_WD_merge_Not_connected.info()

In [ ]:
WDLittbank_WD_merge_Not_connected["librisid"].value_counts()

In [ ]:
WDLittbank_WD_merge_Not_connected[WDLittbank_WD_merge_Not_connected["librisid"] =="20dgkckl01lmd80"]

In [ ]:
WDLittbank_WD_merge_Not_connected["wikidata.sbl_link"].value_counts()

In [ ]:
#WDLittbank_WD_merge_Not_connected["sources"].value_counts()

In [ ]:
#WDLittbank_WD_merge_Not_connected["popularity"].value_counts()

In [ ]:
# WDLittbank_WD_merge_Not_connected["intro_text"].value_counts()